In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import time

#### API-KEY

In [2]:
apikey = "sk-73eAaaMAgCE1wFNy58A29d4849B447CaA2E287A04d7fBfAc"

#### 输出结果函数

In [3]:
## 输出上下游产品的函数
## up
def send_chat_message_up(industry, up="上游", retry=0):
    url = 'http://23.105.199.129:3002/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {apikey}',
        'Content-Type': 'application/json',
    }
    payload = {
        "model" : "gpt-4",
        "messages": [
            {
                "content": f"提供给我{industry}的{up}产品，只需要五个，并用‘、’隔开给我。注意，不要带回答的前缀",
                "role": "user"
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        response_data = response.json()
        # 处理响应数据
        return response_data['choices'][0]['message']['content']
    else:
        print("Request failed with status code:", response.status_code)
        if retry > 10:
            print("重试次数过多", industry)
            return None
        return send_chat_message_up(industry, up="上游", retry=retry + 1)

In [4]:
## down
def send_chat_message_down(industry, up="下游", retry=0):
    url = 'http://23.105.199.129:3002/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {apikey}',
        'Content-Type': 'application/json',
    }
    payload = {
        "model" : "gpt-4",
        "messages": [
            {
                "content": f"提供给我{industry}的{up}产品，只需要五个，并用‘、’隔开给我。注意，不要带回答的前缀",
                "role": "user"
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        response_data = response.json()
        # 处理响应数据
        return response_data['choices'][0]['message']['content']
    else:
        print("Request failed with status code:", response.status_code)
        if retry > 10:
            print("重试次数过多", industry)
            return None
        return send_chat_message_down(industry, up="下游", retry=retry + 1)

In [5]:
# 测试
send_chat_message_up("房地产")

'土地、建筑材料、建筑设备、建筑设计服务、工程施工服务'

### 多线程填充数据

In [6]:
import logging
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import time

# ## 取前十个数据作为测试集
# temp = pd.read_excel(io=r"C:\Users\TUNGEE\Desktop\产业链补充数据TEMP.xlsx")
# # temp = temp.iloc[0:100,:]
# temp

In [7]:
def fill_keywords(row):
    upstream_keywords = send_chat_message_up(row['经营产品'])
    downstream_keywords = send_chat_message_down(row['经营产品'])          
    row['上游关键词'] = upstream_keywords
    row['下游关键词'] = downstream_keywords
    return row

In [16]:
temp = df_empty_keywords
temp

,Unnamed: 0,股票名称,股票全称,社会统一信用代码,经营产品,股票代码,上游关键词,下游关键词
300,300,洽洽食品,洽洽食品股份有限公司,NaN,香瓜子,2557,NaN,NaN
315,315,江苏国信,江苏国信股份有限公司,NaN,箱包革,2608,NaN,NaN
838,838,民生银行,中国民生银行股份有限公司,NaN,银行卡业务,600016,NaN,NaN
852,852,中国医药,中国医药健康产业股份有限公司,NaN,盐酸林可霉素,600056,NaN,NaN
861,861,国投资本,国投资本股份有限公司,NaN,信用业务,600061,NaN,NaN
...,...,...,...,...,...,...,...,...
6305,6305,锦好医疗,惠州市锦好医疗科技股份有限公司,91441300581432767X,助听器,872925,NaN,NaN
6306,6306,锦好医疗,惠州市锦好医疗科技股份有限公司,91441300581432767X,雾化器,872925,NaN,NaN
6307,6307,锦好医疗,惠州市锦好医疗科技股份有限公司,91441300581432767X,气垫床,872925,NaN,NaN
6308,6308,九菱科技,荆州九菱科技股份有限公司,91421000178965352L,粉末冶金产品,873305,NaN,NaN


In [22]:
# 配置日志
logging.basicConfig(filename=r"C:\Users\TUNGEE\Desktop\fill_keywords.log", level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler()  # 创建一个控制台处理程序
logging.getLogger().addHandler(console_handler)  # 将控制台处理程序添加到根记录器

# 创建 tqdm 实例
tqdm_instance = tqdm(total=len(temp), desc="处理行", position=0, leave=True)

def process_row(i, row):
    try:
        # 填充关键词
        keywords_result = fill_keywords(row)
        temp.loc[i] = keywords_result
        # 记录填充关键词的信息到日志

        # gpt输出包含道歉，则出现gpt回答问题出错
        if(keywords_result['上游关键词'].__contains__("对不起") or keywords_result['下游关键词'].__contains__('对不起')):
            logging.info(f"---gpt error: answer error--- 行 {i} 上游关键词结果: {row['上游关键词']} ||  下游关键词结果: {row['下游关键词']}")

        ## gpt输出包含冒号，则出现格式回答问题
        if(keywords_result['上游关键词'].__contains__("：") or keywords_result['下游关键词'].__contains__('：')):
            logging.info(f"---gpt error: prefigx error--- 行 {i} 上游关键词结果: {row['上游关键词']} || 下游关键词结果: {row['下游关键词']}")
    except Exception as e:
        # 记录异常信息到日志
        logging.error(f"填充时出错: {e}")

    # 每处理完一行就保存进度
    temp.to_excel(r"C:\Users\TUNGEE\Desktop\one_temp.xlsx", index=False)

    # 更新进度条
    tqdm_instance.update(1)

# 设置线程数
num_threads = 3

# 使用 ThreadPoolExecutor 进行并发处理
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # 提交每一行的任务以实现并行处理
    futures = [executor.submit(process_row, i, row) for i, row in temp.iterrows()]

    # 等待所有任务完成
    for future in tqdm(futures, desc="等待线程", position=1, leave=True):
        future.result()

# 关闭 tqdm 实例
tqdm_instance.close()


处理行: 100%|██████████| 1/1 [00:07<00:00,  7.11s/it]


In [9]:
temp_2 = pd.read_excel(io=r"C:\Users\TUNGEE\Desktop\temp.xlsx")

In [10]:
temp_2.info

<bound method DataFrame.info of       Unnamed: 0  股票名称             股票全称            社会统一信用代码    经营产品    股票代码  \
0              0  沙河股份       沙河实业股份有限公司                 NaN   房地产开发      14   
1              1  ST深天  深圳市天地(集团)股份有限公司                 NaN   商品混凝土      23   
2              2  ST深天  深圳市天地(集团)股份有限公司                 NaN    水泥制品      23   
3              3  ST深天  深圳市天地(集团)股份有限公司                 NaN  机电设备维修      23   
4              4  深南电A     深圳南山热电股份有限公司                 NaN      供电      37   
...          ...   ...              ...                 ...     ...     ...   
6305        6305  锦好医疗  惠州市锦好医疗科技股份有限公司  91441300581432767X     助听器  872925   
6306        6306  锦好医疗  惠州市锦好医疗科技股份有限公司  91441300581432767X     雾化器  872925   
6307        6307  锦好医疗  惠州市锦好医疗科技股份有限公司  91441300581432767X     气垫床  872925   
6308        6308  九菱科技     荆州九菱科技股份有限公司  91421000178965352L  粉末冶金产品  873305   
6309        6309  九菱科技     荆州九菱科技股份有限公司  91421000178965352L  磁性材料产品  873305   

                   

In [11]:
# 假设你的DataFrame变量名为df
# 选择上游关键词或下游关键词为空的部分
df_empty_keywords = temp_2[(temp_2['上游关键词'].isnull()) | (temp_2['下游关键词'].isnull())]
# 选择上游关键词和下游关键词都不为空的部分
df_both_keywords = temp_2.dropna(subset=['上游关键词', '下游关键词'])
df_both_keywords = temp_2.dropna()

In [12]:
## 找出空缺的
df_empty_keywords.to_excel(r"C:\Users\TUNGEE\Desktop\empty_keywords.xlsx")
## 找出填充完的
df_both_keywords.to_excel(r"C:\Users\TUNGEE\Desktop\both_keywords.xlsx")

In [14]:
df_empty_keywords

,Unnamed: 0,股票名称,股票全称,社会统一信用代码,经营产品,股票代码,上游关键词,下游关键词
300,300,洽洽食品,洽洽食品股份有限公司,NaN,香瓜子,2557,NaN,NaN
315,315,江苏国信,江苏国信股份有限公司,NaN,箱包革,2608,NaN,NaN
838,838,民生银行,中国民生银行股份有限公司,NaN,银行卡业务,600016,NaN,NaN
852,852,中国医药,中国医药健康产业股份有限公司,NaN,盐酸林可霉素,600056,NaN,NaN
861,861,国投资本,国投资本股份有限公司,NaN,信用业务,600061,NaN,NaN
...,...,...,...,...,...,...,...,...
6305,6305,锦好医疗,惠州市锦好医疗科技股份有限公司,91441300581432767X,助听器,872925,NaN,NaN
6306,6306,锦好医疗,惠州市锦好医疗科技股份有限公司,91441300581432767X,雾化器,872925,NaN,NaN
6307,6307,锦好医疗,惠州市锦好医疗科技股份有限公司,91441300581432767X,气垫床,872925,NaN,NaN
6308,6308,九菱科技,荆州九菱科技股份有限公司,91421000178965352L,粉末冶金产品,873305,NaN,NaN


In [15]:
df_both_keywords

,Unnamed: 0,股票名称,股票全称,社会统一信用代码,经营产品,股票代码,上游关键词,下游关键词
0,0,沙河股份,沙河实业股份有限公司,NaN,房地产开发,14,土地资源、建筑设计服务、建材供应、基础设施建设服务、施工服务,购房贷款服务、物业管理服务、家居装修服务、家居家电销售、房屋买卖中介服务
1,1,ST深天,深圳市天地(集团)股份有限公司,NaN,商品混凝土,23,水泥、砂子、石头、钢筋、添加剂,道路、桥梁、大楼、隧道、停车场
2,2,ST深天,深圳市天地(集团)股份有限公司,NaN,水泥制品,23,石灰石、高炉渣、水、煤炭、石膏,建筑结构、桥梁、道路、水泥管道、预制混凝土结构
3,3,ST深天,深圳市天地(集团)股份有限公司,NaN,机电设备维修,23,机械配件、电子元件、测量工具、润滑油、保护工具,维修工具、更换零部件、维修后的设备、设备维护材料、保修服务
4,4,深南电A,深圳南山热电股份有限公司,NaN,供电,37,电力发电机、燃料（如煤、天然气或水力）、输电线路、变压器、电表,电脑、冰箱、电视、空调、电灯
...,...,...,...,...,...,...,...,...
4510,4510,汉马科技,汉马科技集团股份有限公司,913405007139577931,专用车,600375,汽车零部件、汽车发动机、汽车电池、汽车涂料、汽车座椅,出租车服务、送货车服务、高端租赁车服务、长途客车服务、私人拼车服务
4511,4511,汉马科技,汉马科技集团股份有限公司,913405007139577931,汽车零部件,600375,钢材、塑料、橡胶、电子元器件、铝材,汽车轮胎、汽车座椅、汽车头灯、汽车刹车系统、汽车导航系统
4512,4512,宁沪高速,江苏宁沪高速公路股份有限公司,91320000134762764K,路桥建设,600377,混凝土、钢材、运输设备、建筑机械、电气设备,交通信号设备、安全防护设备、路灯及照明设备、交通标线涂料、桥梁检测仪器
4513,4513,宁沪高速,江苏宁沪高速公路股份有限公司,91320000134762764K,路桥经营,600377,钢材、混凝土、砂石、建筑机械设备、公路建设工程设计方案,物流运输、旅游业、房地产开发、城市规划设计、零售业


In [21]:
df_temp = temp
temp = temp[
    (temp['上游关键词'].isnull()) | (temp['下游关键词'].isnull())
    ]

### 股票代码填充0

In [25]:
fin = pd.read_excel(io=r"C:\Users\TUNGEE\Desktop\补充数据-上下游补充.xlsx")
fin.info

<bound method DataFrame.info of       股票名称             股票全称            社会统一信用代码    经营产品    股票代码  \
0     ST星源     深圳世纪星源股份有限公司  914403006188470942    酒店经营       5   
1     ST星源     深圳世纪星源股份有限公司  914403006188470942    物业管理       5   
2     沙河股份       沙河实业股份有限公司                 NaN   房地产开发      14   
3     ST深天  深圳市天地(集团)股份有限公司                 NaN   商品混凝土      23   
4     ST深天  深圳市天地(集团)股份有限公司                 NaN    水泥制品      23   
...    ...              ...                 ...     ...     ...   
6306  七丰精工     七丰精工科技股份有限公司                 NaN     高强度  873169   
6307  七丰精工     七丰精工科技股份有限公司                 NaN   耐低温螺栓  873169   
6308  七丰精工     七丰精工科技股份有限公司                 NaN    自锁螺母  873169   
6309  九菱科技     荆州九菱科技股份有限公司  91421000178965352L  粉末冶金产品  873305   
6310  九菱科技     荆州九菱科技股份有限公司  91421000178965352L  磁性材料产品  873305   

                                    上游关键词  \
0                酒店用品、餐饮原料、装修建材、电子设备、清洁用品   
1     物业管理软件、安全监控设备、楼宇自动化系统、清洁维护设备、智能门禁系统   
2          土地资源、建筑设计服务、建材供应、